## Getting Started

In [1]:
import time
import numpy as np
from fourier_grx_client import RobotClient, ControlGroup
from rich.table import Table
from rich.console import Console
console = Console()
print = console.print
np.set_printoptions(precision=3, suppress=True)

### Start the server on the robot

Grx server must be started on the robot torso computer before running the client and sending commands. To start the server on the robot, you need to follow these steps:

1. Place the robot in an appropriate position, especially the arms, and switch on the E-stop.

2. Start the server on the robot by running the following command on the robot:

```bash
grx run ./path/to/config --namesapce gr/my_awesome_robot
```
Replace `./path/to/config` with the path to your configuration file. Default configuration files are provided under 'config' directory.

Namespace is used to identify the robot when connecting to it from the client. You can replace `my_awesome_robot` with any name you like.





If the server is successfully started, you should see the following message in the terminal:

```
2024-09-14 16:32:57 | INFO | #################################
2024-09-14 16:32:57 | SUCCESS | RobotServer OK!



### Start the client 

After the server is successfully started, you can start the client and connect to the server. In this example, we will assume that the client is running on the same machine as the server. If the client is running on a different machine, you need to specify the IP address of the server.

The namespace needs to be aligned with the namespace of that when starting the server. 


In [2]:
client= RobotClient(namespace="gr/my_awesome_robot", server_ip="localhost")

2024-09-13 18:46:53.041 | INFO     | fourier_grx_client.client:__init__:71 - RobotClient starting...
2024-09-13 18:46:54.250 | SUCCESS  | fourier_grx_client.client:__init__:157 - RobotClient started with namespace: gr/my_awesome_robot


## Important Concepts

### Namespace

We use the concept of namespace to avoid name conflicts between different robots.
A namespace is a way to group related topics together. It is a prefix that is added to the topic name. For example, if the namespace is `gr/my_awesome_robot`, then the topic name `control` will become `gr/my_awesome_robot/control`.

### Joint Orders

The robot has 32 joints. All internal representations of the robot's state, commands, and parameters are in the order of these joints. 

The joint orders for `GR1T2` are as follows:

| Index | Joint Name |
| --- | --- |
| 0 | left_hip_roll_joint |
| 1 | left_hip_yaw_joint |
| 2 | left_hip_pitch_joint |
| 3 | left_knee_pitch_joint |
| 4 | left_ankle_pitch_joint |
| 5 | left_ankle_roll_joint |
| 6 | right_hip_roll_joint |
| 7 | right_hip_yaw_joint |
| 8 | right_hip_pitch_joint |
| 9 | right_knee_pitch_joint |
| 10 | right_ankle_pitch_joint |
| 11 | right_ankle_roll_joint |
| 12 | waist_yaw_joint |
| 13 | waist_pitch_joint |
| 14 | waist_roll_joint |
| 15 | head_pitch_joint |
| 16 | head_roll_joint |
| 17 | head_yaw_joint |
| 18 | left_shoulder_pitch_joint |
| 19 | left_shoulder_roll_joint |
| 20 | left_shoulder_yaw_joint |
| 21 | left_elbow_pitch_joint |
| 22 | left_wrist_yaw_joint |
| 23 | left_wrist_roll_joint |
| 24 | left_wrist_pitch_joint |
| 25 | right_shoulder_pitch_joint |
| 26 | right_shoulder_roll_joint |
| 27 | right_shoulder_yaw_joint |
| 28 | right_elbow_pitch_joint |
| 29 | right_wrist_yaw_joint |
| 30 | right_wrist_roll_joint |
| 31 | right_wrist_pitch_joint |


### Absolute encoder and `sensor_offset.json`

The robot has absolute encoders for joints 0 to 14, AKA the leg joints and the waist joints. The absolute encoders are used to determine the absolute position of the joints. The calibration file `sensor_offset.json` contains the offset values for the absolute encoders. The offset values are used to calibrate the absolute encoders. The calibration file is specific to each robot. Before running the robot, make sure that the calibration file is present in the working directory where the `grx` server is running. 

## Usage

### Calibration

For arm and head actuators, the home positons are determiend when switching on the E-stop button, which means you need to place them properly every time you power on the actuators.

For waist and leg actuators, the home positions are determined according to the `sensor_offset.json` file. This file is generated by the calibration process and contains the offsets of the encoders. You only need to run the calibration process once, and the `sensor_offset.json` file will be saved to the working directory where the `grx` server is running.

Running the following command will execute the calibration of the absolute encoders:

```bash
grx calibrate --namespace gr/my_awesome_robot --ip 192.168.x.x
```

Or if you prefer to do it by code, you can use the following code snippet:

```python
client.calibrate_sensors()
```



### Getting the robot states

The robot states can be accessed on the following properties:

In [3]:
print(f"{client.joint_positions=}")
print(f"{client.joint_velocity=}")
print(f"{client.joint_current=}")
print(f"{client.joint_effort=}")

client.joint_positions=array([ 0.   ,  0.   , -0.   ,  0.001,  0.   , -0.001,  0.   ,  0.   ,
       -0.001,  0.001,  0.   ,  0.001,  0.   ,  0.   ,  0.   ,  0.001,
       -0.001, -0.061, -0.004, -0.027, -0.   ,  0.   ,  0.   ,  0.032,
        0.05 , -0.09 ,  0.007,  0.   ,  0.   , -0.178,  0.045, -0.015])

client.joint_velocity=array([-0.001, -0.002, -0.008,  0.007, -0.001,  0.003,  0.001,  0.001,
        0.001, -0.001, -0.001,  0.003, -0.   ,  0.001, -0.002,  0.001,
       -0.004, -0.   ,  0.001, -0.   ,  0.   , -0.   ,  0.001,  0.002,
        0.001,  0.   , -0.   ,  0.   , -0.   , -0.001, -0.   , -0.001])

client.joint_current=array([ 0.023, -0.026, -0.024,  0.225,  0.002, -0.006,  0.007, -0.05 ,
        0.108, -0.151, -0.004, -0.005,  0.002,  0.021, -0.005, -0.002,
       -0.014, -0.019, -0.001,  0.01 ,  0.008,  0.02 ,  0.006, -0.003,
       -0.026,  0.006,  0.004,  0.005, -0.002,  0.004,  0.005,  0.019])

client.joint_effort=array([ 0.087, -0.089, -0.043,  0.409, -0.009,  0.009,  0.027, -0.17 ,
        0.197, -0.274, -0.02 , -0.001,  0.008,  0.072, -0.019, -0.049,
        0.024, -0.059, -0.003,  0.049,  0.05 ,  0.117,  0.012,  0.043,
       -0.03 ,  0.029,  0.021,  0.028, -0.012,  0.007,  0.026, -0.025])

The above are the convenient methods to access the robot states. However, the robot states can also be accessed directly from the `client.states` dictionary. We can inspect it to find the available states:

In [4]:
table = Table("Type", "Data", title="Current :robot: states (in radians)")
for sensor_type, sensor_data in client.states.items():
    for sensor_name, sensor_reading in sensor_data.items():
        table.add_row(
            sensor_type + "/" + sensor_name,
            str(np.deg2rad(sensor_reading)),
        )
print(table)

                                  Current 🤖 states (in radians)                                  
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Type                  ┃ Data                                                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ imu/quat              │ [-0. -0. -0.  0.]                                                      │
│ imu/euler_angle       │ [ 3.142 -0.     0.   ]                                                 │
│ imu/angular_velocity  │ [ 0.  0. -0.]                                                          │
│ imu/acceleration      │ [-0. -0.  0.]                                                          │
│ joint/position        │ [ 0.     0.    -0.     0.001  0.    -0.001  0.     0.    -0.001  0.001 │
│                       │   0.     0.001  0.     0.     0.     0.001 -0.001 -0.061 -0.004 -0.027 │
│                       │   0.     0.     0.     0.032  0.05  -0.09   0.007  0.     0.    -0.178 │
│                       │   0.045 -0.015]                                                        │
│ joint/velocity        │ [-0.001 -0.003  0.013  0.005  0.     0.002  0.001  0.    -0.013 -0.006 │
│                       │   0.001 -0.001 -0.     0.002 -0.003 -0.001  0.001  0.     0.001  0.    │
│                       │   0.     0.     0.002  0.     0.002  0.     0.     0.     0.     0.004 │
│                       │   0.    -0.001]                                                        │
│ joint/effort          │ [ 0.002 -0.002 -0.001  0.007 -0.     0.     0.    -0.003  0.003 -0.005 │
│                       │  -0.    -0.     0.     0.001 -0.    -0.001  0.    -0.001 -0.     0.001 │
│                       │   0.001  0.002  0.     0.001 -0.001  0.001  0.     0.    -0.     0.    │
│                       │   0.    -0.   ]                                                        │
│ joint/current         │ [ 0.    -0.    -0.     0.004  0.    -0.     0.    -0.001  0.002 -0.003 │
│                       │  -0.    -0.     0.     0.    -0.    -0.    -0.    -0.    -0.     0.    │
│                       │   0.     0.     0.    -0.    -0.     0.     0.     0.    -0.     0.    │
│                       │   0.     0.   ]                                                        │
│ base/estimate_xyz     │ [0.    0.    0.016]                                                    │
│ base/estimate_xyz_vel │ [0. 0. 0.]                                                             │
└───────────────────────┴────────────────────────────────────────────────────────────────────────┘

### Enabling and disabling the robot

Now, to actually drive the robot, we need to first enable it:

In [5]:
client.enable()

After enabling the robot, you can hear the robot's motors turning on. To disable it, just run:

```python
client.disable()
```

In [6]:
client.disable()

### Setting and getting joint gains

Before we can control the robot, we need to set the joint gains. The joint gains are the parameters that control how the robot's joints respond to the commands. The joint gains can be set using the `client.set_gains` method. The method takes the following arguments:

```python
def set_gains(
        self,
        position_control_kp: list[float] | None = None,
        velocity_control_kp: list[float] | None = None,
        velocity_control_ki: list[float] | None = None,
        pd_control_kp: list[float] | None = None,
        pd_control_kd: list[float] | None = None,
    ):
    ...
``` 
Where each of the arguments is a list of floats representing the gains for each joint. The gains are in the same order as the joints in the robot. For example, to set the position control gains kp for all joints to 1.0, you can run:

```python
client.set_gains(position_control_kp=[1.0]*32)
```

You can get the gains using the 'get_gains' method. 

Here's an example:

In [ ]:
print(client.get_gains())

### Moving joint positions

After enabling the actuators, we can move the joints to a desired position using the `move_joints()` method.

The method takes the following arguments:
```python
    def move_joints(
        self,
        group: ControlGroup | list | str,
        positions: np.ndarray | list,
        duration: float = 0.0,
        degrees: bool = False,
        blocking: bool = True,
    ):
```
Args:
    group (ControlGroup | list | str): The group of joints to move, specified by a string or a ControlGroup enum, or a list of joint indices.
    positions (np.ndarray[float]): target joint position in degrees.
    duration (float, optional): Time duration in seconds. If set to 0, the joints will move in their maximum speed without interpolation. Defaults to 0.0.
    degrees (bool, optional): Whether the joint positions are in degrees. Defaults to False.
    blocking (bool, optional): If True, block until the move is completed. Defaults to True.

Here's some examples of moving the joints to a position:


In [ ]:

client.move_joints(ControlGroup.LEFT_ARM, [0, 0, 0, 20, 0, 0, 0], duration=2, degrees=True)

In [ ]:
client.move_joints([23, 24], [0.17, 0.17], degrees=False)

In [ ]:
client.move_joints("left_arm", [0, 0, 0, 20, 0, 0, 0], degrees=True)